In [1]:
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q datasets
!pip install -q pandas
!pip install -q torch
!pip install -q torchinfo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00

In [2]:
import evaluate
import pandas as pd
import numpy as np
import torch

from transformers import BertTokenizer, AutoTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments, pipeline
from torchinfo import summary
from datasets import Dataset, load_dataset
from pprint import pprint

In [3]:
ds = load_dataset("rony/climate-change-MRC")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


README.md:   0%|          | 0.00/633 [00:00<?, ?B/s]

CCMRC_train.json:   0%|          | 0.00/12.8M [00:00<?, ?B/s]

CCMRC_validation.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

CCMRC_test.json:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [4]:
def prepare_data(data):
  articles = []

  for paragraph in data:
    context = paragraph['context']
    for qa in paragraph['qas']:
      question = qa['question']
      id = qa['id']
      for ans in qa['answers']:
        answer = ans['text']
        answer_start = ans['answer_start']
        articles.append({'context': context, 'question': question, 'id': id, 'answer': answer, 'answer_start': answer_start})

  return articles

In [5]:
train_ds = ds["train"]
valid_ds = ds["validation"]
test_ds = ds["test"]

# each is a 1-item list, so take first index
train_ds = train_ds[0]
valid_ds = valid_ds[0]
test_ds = test_ds[0]

# take the 'data' key of the dict, ignoring 'version' (there's just one)
train_ds = train_ds['data'][0]['paragraphs']
valid_ds = valid_ds['data'][0]['paragraphs']
test_ds = test_ds['data'][0]['paragraphs']
# each dataset is a list of dicts, where each list item is a context paragraph ('context' key) with qas ('qas' key) which contain questions, id, and answer

train_df = pd.DataFrame(prepare_data(train_ds))
print(f"{train_df.shape=}")

valid_df = pd.DataFrame(prepare_data(valid_ds))
print(f"{valid_df.shape=}")

test_df = pd.DataFrame(prepare_data(test_ds))
print(f"{test_df.shape=}")

train_df.shape=(14756, 5)
valid_df.shape=(4229, 5)
test_df.shape=(2096, 5)


In [6]:
train_df.head()

,context,question,id,answer,answer_start
0,"outside of the clean air act, there is support...",Is a mandate for electric production to come f...,665,"and there is ongoing interest in a ""national r...",222
1,"outside of the clean air act, there is support...",How wide ranging are climate policies across t...,666,"as of 2010, climate policies were being contem...",618
2,"outside of the clean air act, there is support...",What kind of energy standards have been implem...,667,"at latest count, 30 states have implemented re...",734
3,acknowledgments. we thank jim haywood for the ...,What was Jim Haywood thanked for?,1731,we thank jim haywood for the aerosol forcing pdf,17
4,acknowledgments. we thank jim haywood for the ...,Where was Jonathan Gregory supported?,1732,jonathan gregory was supported at the universi...,277


In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def get_answer_from_model(context, question):

    inputs = tokenizer(question, context, truncation=True, return_tensors='pt')

    outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    answer_tokens = inputs['input_ids'][0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_tokens)

    return answer

# apply on validation data
valid_df.loc[:, 'bert_answer'] = valid_df.apply(lambda row: get_answer_from_model(row['context'], row['question']), axis=1)

# apply on subset of validation data
# valid_subset.loc[:, 'predicted_answer'] = valid_subset.apply(lambda row: get_answer_from_model(row['context'], row['question']), axis=1)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [9]:
valid_df.to_parquet('valid_bert_scored.parquet')
valid_df.head()

,context,question,id,answer,answer_start,bert_answer
0,climate change has serious implications for fo...,State one of the climatic changes that can cau...,14850,changes in the ranges of agricultural pests an...,600,
1,climate change has serious implications for fo...,Mention some implication of climatic changes o...,14851,"serious implications for food production, proc...",19,
2,the data for the research was obtained from a ...,How were the research data obtained? They were...,7397,the data for the research was obtained from a ...,0,research data obtained? they were obtained fro...
3,the data for the research was obtained from a ...,The survey includes which districts include Ya...,7398,the data for the research was obtained from a ...,0,households were selected randomly proportional...
4,the data for the research was obtained from a ...,How were the specific study sites selected? Th...,7399,the specific study sites within the districts ...,180,households were selected randomly proportional...


In [10]:
# read in BERT answers
valid_df = pd.read_parquet('valid_bert_scored.parquet')

# evaluate BERT answers
rouge = evaluate.load('rouge')
predictions = valid_df['bert_answer'].tolist()
references = valid_df['answer'].tolist()
results = rouge.compute(predictions=predictions,
                       references=references)
print('validation \n',results)

validation 
 {'rouge1': 0.14298222290550422, 'rouge2': 0.09680856469080434, 'rougeL': 0.12396625291133445, 'rougeLsum': 0.1239123836424291}


In [11]:
test_df.loc[:, 'bert_answer'] = test_df.apply(lambda row: get_answer_from_model(row['context'], row['question']), axis=1)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [13]:
test_df.to_parquet('test_bert_scored.parquet')
test_df.head()

,context,question,id,answer,answer_start,bert_answer
0,some more detailed work has been done at natio...,The 9 percent reduction of rice in Bangladesh ...,14095,flooding damage and climate variability,514,
1,some more detailed work has been done at natio...,What kind of model of Bangladesh was had been ...,14096,a dynamic economywide model,70,
2,some more detailed work has been done at natio...,What approach did Ahmed use to estimate how ch...,14097,a modelling approach,639,
3,extreme sea level height fluctuations are also...,Where height fluctuations are large?,2843,extreme sea level height fluctuations are also...,0,
4,extreme sea level height fluctuations are also...,How non-tide sea levels are obtained?,2844,the non-tide sea levels are obtained by spectr...,167,


In [16]:
# read in BERT answers
test_df = pd.read_parquet('test_bert_scored.parquet')

# evaluate BERT answers
rouge = evaluate.load('rouge')
predictions = test_df['bert_answer'].tolist()
references = test_df['answer'].tolist()
results = rouge.compute(predictions=predictions,
                       references=references)
print('test \n',results)

test 
 {'rouge1': 0.13808890794923134, 'rouge2': 0.09449441091673152, 'rougeL': 0.1206936809408421, 'rougeLsum': 0.12057938002898017}


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize function
def preprocess_function(examples):
    tokenized_inputs = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        padding="max_length",
        max_length=512,
        return_overflowing_tokens=True
    )

    start_positions = []
    end_positions = []

    for i in range(len(examples["answer"])):
        context = examples["context"][i]
        answer = examples["answer"][i]
        answer_start = examples["answer_start"][i]

        answer_end = answer_start + len(answer) - 1

        start_token = tokenizer.encode(context[:answer_start], add_special_tokens=False)
        end_token = tokenizer.encode(context[:answer_end + 1], add_special_tokens=False)

        start_positions.append(len(start_token))
        end_positions.append(len(end_token) - 1)

    tokenized_inputs["start_positions"] = start_positions
    tokenized_inputs["end_positions"] = end_positions

    return tokenized_inputs

# apply preprocess function to datasets for tokenization
tokenized_train = Dataset.from_pandas(train_df).map(preprocess_function, batched=True)
tokenized_valid = Dataset.from_pandas(valid_df).map(preprocess_function, batched=True)
tokenized_test = Dataset.from_pandas(test_df).map(preprocess_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/14756 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/4229 [00:00<?, ? examples/s]

Map:   0%|          | 0/2096 [00:00<?, ? examples/s]

In [ ]:
# Load the BERT model for question answering
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Prepare the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid
)

# Train the model
# trainer.train()

# Evaluate the model on the validation set
results = trainer.evaluate()
print(results)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'eval_loss': 6.242977142333984, 'eval_model_preparation_time': 0.0051, 'eval_runtime': 128.8052, 'eval_samples_per_second': 32.833, 'eval_steps_per_second': 2.057}


In [ ]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

bleu_metric = evaluate.load("bleu")
bleurt_metric = evaluate.load("bleurt")
rouge_metric = evaluate.load("rouge")
